# Track Azure Databricks ML experiments with MLflow and Azure Machine Learning 
This notebook shows how we can train a classification model in Azure Databricks.

In [ ]:
import mlflow

region = "<AML workspace region>"
subscription_id = "<subscription id>"
resource_group = "<AML resource group>"
workspace_name = "<AML workspace name>"

azureml_mlflow_uri = f"azureml://{region}.api.azureml.ms/mlflow/v1.0/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/Microsoft.MachineLearningServices/workspaces/{workspace_name}"
mlflow.set_tracking_uri(azureml_mlflow_uri)

# Set the experiment name

In [ ]:
mlflow.set_experiment(experiment_name="heart-condition-classifier")

In [ ]:

import pandas as pd

file_url = "http://storage.googleapis.com/download.tensorflow.org/data/heart.csv"
df = pd.read_csv(file_url)
display(df)


In [ ]:
df["thal"] = df["thal"].astype("category").cat.codes

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.drop("target", axis=1), df["target"], test_size=0.3
)

#  Use MLflow in Azure Databricks in the same way as you're used to

In [ ]:
mlflow.xgboost.autolog()

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, recall_score

model = XGBClassifier(use_label_encoder=False, eval_metric="logloss")

In [ ]:
with mlflow.start_run() as run:
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)

    print("Accuracy: %.2f%%" % (accuracy * 100.0))
    print("Recall: %.2f%%" % (recall * 100.0))

#  Register the MLFlow model in AML model registery

In [ ]:
mlflow.register_model(
    model_uri=f"runs:/{run.info.run_id}/model", name="databricks-heart-classifier"
)